# tutorial 45,Import custom vm Image as Ami Using boto3 python

In [110]:
import boto3
s3_client=boto3.client("s3")

In [113]:
#upload custom vhd image to s3 bucket
s3_client.upload_file(Filename="/Users/roni/Downloads/64bit/hd.vhd",
    Bucket="tutorial45",
    Key="tutorial45.vhd"
    )

In [114]:
#create iam role
iam_client=boto3.client("iam")



In [116]:
import json
s=json.dumps({
 "Version": "2012-10-17",
 "Statement": [
 {
 "Effect": "Allow",
 "Principal": { "Service": "vmie.amazonaws.com" },
 "Action": "sts:AssumeRole",
 "Condition": {
 "StringEquals":{
 "sts:Externalid": "vmimport"
 }
 }
 }
 ]
})

In [117]:
iam_client.create_role(
      RoleName='vmimport',
      AssumeRolePolicyDocument=s,
      Description='vmimport tutorial45',
      MaxSessionDuration=3600,
      Tags=[
          {
              'Key': 'Name',
              'Value': 'vmimport'
          },
      ])

{'Role': {'Path': '/',
  'RoleName': 'vmimport',
  'RoleId': 'AROAVYGKL34QRCKONZNDO',
  'Arn': 'arn:aws:iam::395561393953:role/vmimport',
  'CreateDate': datetime.datetime(2020, 12, 19, 18, 32, 23, tzinfo=tzutc()),
  'AssumeRolePolicyDocument': {'Version': '2012-10-17',
   'Statement': [{'Effect': 'Allow',
     'Principal': {'Service': 'vmie.amazonaws.com'},
     'Action': 'sts:AssumeRole',
     'Condition': {'StringEquals': {'sts:Externalid': 'vmimport'}}}]},
  'Tags': [{'Key': 'Name', 'Value': 'vmimport'}]},
 'ResponseMetadata': {'RequestId': '47bfbe22-c899-4a75-aeaf-70137ab09fe7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '47bfbe22-c899-4a75-aeaf-70137ab09fe7',
   'content-type': 'text/xml',
   'content-length': '995',
   'date': 'Sat, 19 Dec 2020 18:32:23 GMT'},
  'RetryAttempts': 0}}

In [ ]:
s1=json.dumps({
 "Version":"2012-10-17",
 "Statement":[
 {
 "Effect": "Allow",
 "Action": [
 "s3:GetBucketLocation",
 "s3:GetObject",
 "s3:ListBucket"
 ],
 "Resource": [
 "arn:aws:s3:::tutorial45",
 "arn:aws:s3:::tutorial45/*"
 ]
 },
 {
 "Effect": "Allow",
 "Action": [
 "s3:GetBucketLocation",
 "s3:GetObject",
 "s3:ListBucket",
 "s3:PutObject",
 "s3:GetBucketAcl"
 ],
 "Resource": [
 "arn:aws:s3:::tutorial45",
 "arn:aws:s3:::tutorial45/*"
 ]
 },
 {
 "Effect": "Allow",
 "Action": [
 "ec2:ModifySnapshotAttribute",
 "ec2:CopySnapshot",
 "ec2:RegisterImage",
 "ec2:Describe*"
 ],
 "Resource": "*")

In [119]:
#put inline policy for the iam role
iam_client.put_role_policy(RoleName='vmimport',
      PolicyName='tutorial45',
      PolicyDocument=s1)


{'ResponseMetadata': {'RequestId': 'be3957a4-d4fc-45e5-85ae-c36be1a4ec51',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'be3957a4-d4fc-45e5-85ae-c36be1a4ec51',
   'content-type': 'text/xml',
   'content-length': '206',
   'date': 'Sat, 19 Dec 2020 18:35:55 GMT'},
  'RetryAttempts': 0}}

In [120]:
#import the custom vm image as ami
ec2_client=boto3.client("ec2")

In [123]:
ec2_client.import_image(DiskContainers=[
          {'Description': 'tutorial45_custom_ami',
              'DeviceName': '/dev/xvda',
              'Format': 'vhd',
              'UserBucket': {
                  'S3Bucket': 'tutorial45',
                  'S3Key': 'tutorial45.vhd'
              }
          },
      ],)

{'ImportTaskId': 'import-ami-09448588081a28b39',
 'Progress': '1',
 'SnapshotDetails': [{'Description': 'tutorial45_custom_ami',
   'DeviceName': '/dev/xvda',
   'DiskImageSize': 0.0,
   'Format': 'VHD',
   'UserBucket': {'S3Bucket': 'tutorial45', 'S3Key': 'tutorial45.vhd'}}],
 'Status': 'active',
 'StatusMessage': 'pending',
 'ResponseMetadata': {'RequestId': '5a2a471e-2616-4508-a001-89a8e04223a4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5a2a471e-2616-4508-a001-89a8e04223a4',
   'content-type': 'text/xml;charset=UTF-8',
   'transfer-encoding': 'chunked',
   'vary': 'accept-encoding',
   'date': 'Sat, 19 Dec 2020 18:40:34 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [158]:
ec2_client.describe_import_image_tasks(ImportTaskIds=[
          'import-ami-09448588081a28b39'
      ],)

{'ImportImageTasks': [{'Architecture': 'x86_64',
   'ImageId': 'ami-097f56aeb7ccae900',
   'ImportTaskId': 'import-ami-09448588081a28b39',
   'LicenseType': 'BYOL',
   'Platform': 'Linux',
   'SnapshotDetails': [{'Description': 'tutorial45_custom_ami',
     'DeviceName': '/dev/sda1',
     'DiskImageSize': 2813010432.0,
     'Format': 'VHD',
     'SnapshotId': 'snap-01be490167b61388a',
     'Status': 'completed',
     'UserBucket': {'S3Bucket': 'tutorial45', 'S3Key': 'tutorial45.vhd'}}],
   'Status': 'completed',
   'Tags': []}],
 'ResponseMetadata': {'RequestId': '41bea4c6-22ac-46cb-91d5-33844229ce9e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '41bea4c6-22ac-46cb-91d5-33844229ce9e',
   'content-type': 'text/xml;charset=UTF-8',
   'transfer-encoding': 'chunked',
   'vary': 'accept-encoding',
   'date': 'Sat, 19 Dec 2020 19:04:25 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}